In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [ ]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [ ]:
cdatasub = pd.read_csv("london_flows_index.csv",index_col=0)
for index, row in cdatasub.iterrows():
    if row["distance"]==0 or row["population"]==0 or row["jobs"]==0 or row["flows"]==0:
        cdatasub.drop(index, inplace=True)


In [ ]:


x_variables = ["jobs","distance",]
log_x_vars = []
for x in x_variables:
    cdatasub[f"log_{x}"] = np.log(cdatasub[x])
    log_x_vars.append(f"log_{x}")



#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ station_origin+ log_jobs + log_distance-1'
#run a production constrained sim
doubleSim = smf.glm(formula = formula, data=cdatasub, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(doubleSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                43945
Model:                            GLM   Df Residuals:                    43545
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.1409e+05
Date:                Mon, 09 May 2022   Deviance:                   1.6560e+06
Time:                        10:04:01   Pearson chi2:                 2.41e+06
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [ ]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(doubleSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "")
#then once you have done this you can join them back into the dataframes
cdatasub = cdatasub.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
cdatasub.drop(columns = ["coef"], inplace = True)
#check this has worked
cdatasub.head(2)


C:\Users\1\AppData\Local\Temp\ipykernel_47324\402350020.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  coefs["coef"] = coefs["coef"].str.replace(x, "")


,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,alpha_i
0,Abbey Road,Beckton,1,599,442,8510.121774,6.091310,9.049012,3.270351
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.499787,8.236275,3.270351


In [ ]:
D_j = pd.DataFrame(cdatasub.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
cdatasub = cdatasub.merge(D_j, on = "station_destination", how = "left" )
#cdatasub.head(10)

O_i = pd.DataFrame(cdatasub.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
cdatasub = cdatasub.merge(O_i, on = "station_origin", how = "left" )
#cdatasub.head(10)


In [ ]:
alpha_i = doubleSim.params[0:-2]
gamma = doubleSim.params[-2]
beta = -doubleSim.params[-1]

In [ ]:
alpha_i

station_origin[Abbey Road]          3.270351
station_origin[Acton Central]       5.008886
station_origin[Acton Town]          4.397394
station_origin[Aldgate]             3.361125
station_origin[Aldgate East]        3.408728
                                      ...   
station_origin[Wood Street]         5.672160
station_origin[Woodford]            4.955425
station_origin[Woodgrange Park]     5.320215
station_origin[Woodside Park]       4.496709
station_origin[Woolwich Arsenal]    6.701868
Length: 398, dtype: float64

In [ ]:
gamma

0.7301699265801902

In [ ]:
beta

0.8151874614763732

In [ ]:
cdatasub["prodsimest1"] = np.exp(cdatasub["alpha_i"]+gamma*cdatasub["log_jobs"] 
                                 - beta*cdatasub["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
#cdatasub.head(10)

In [ ]:
#first round the estimates
cdatasub["prodsimest1"] = round(cdatasub["prodsimest1"],0)


In [ ]:
RSquared=CalcRSquared(cdatasub["flows"], cdatasub["prodsimest1"])
print("RSquared is",RSquared)

RSquared is 0.39372592328634215


In [ ]:

RMSE=CalcRMSE(cdatasub["flows"], cdatasub["prodsimest1"])
print(RMSE)

120.147


We can examine how the constraints hold for destinations this time:

In [ ]:

def new_sal(row):
    if row["station_destination"] == "Canary Wharf":
        val = row["jobs"]*1.5
    else:
        val = row["jobs"]
    return val
        
cdatasub["Dj3_destsalScenario"] = cdatasub.apply(new_sal, axis =1)
cdatasub.head(10)


,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,alpha_i,D_j,O_i,prodsimest1,Dj3_destsalScenario
0,Abbey Road,Beckton,1,599,442,8510.121774,6.091310,9.049012,3.270351,442,599,1.0,442.0
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.499787,8.236275,3.270351,665,599,4.0,665.0
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,8.534348,3.270351,58772,599,76.0,88158.0
3,Abbey Road,Canning Town,37,599,15428,2228.923167,9.643939,7.709274,3.270351,15428,599,56.0,15428.0
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.096721,8.807842,3.270351,1208,599,4.0,1208.0
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466228,9.048280,3.270351,1748,599,4.0,1748.0
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.745236,8.784484,3.270351,850,599,3.0,850.0
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.415097,8.283576,3.270351,611,599,3.0,611.0
8,Abbey Road,East India,2,599,1522,3384.141666,7.327781,8.126856,3.270351,1522,599,7.0,1522.0
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.538140,8.949793,3.270351,691,599,2.0,691.0


In [ ]:
Dj3_gamma = cdatasub["Dj3_destsalScenario"]**gamma
dist_beta = cdatasub["distance"]**beta
#calcualte the first stage of the Ai values
cdatasub["Ai1"] = Dj3_gamma * dist_beta
#now do the sum over all js bit
A_i = pd.DataFrame(cdatasub.groupby(["station_origin"])["Ai1"].agg(np.sum))
#now divide into 1
A_i["Ai1"] = 1/A_i["Ai1"]
A_i.rename(columns={"Ai1":"A_i2"}, inplace=True)
#and write the A_i values back into the dataframe
cdatasub = cdatasub.merge(A_i, left_on="station_origin", right_index=True, how="left")

In [ ]:
cdatasub["prodsimest4"] = cdatasub["A_i2"]*cdatasub["O_i"]*Dj3_gamma*dist_beta
#round
cdatasub["prodsimest4"] = round(cdatasub["prodsimest4"])

In [ ]:
cdatasub.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_jobs,log_distance,alpha_i,D_j,O_i,prodsimest1,Dj3_destsalScenario,Ai1,A_i2,prodsimest4
0,Abbey Road,Beckton,1,599,442,8510.121774,6.091310,9.049012,3.270351,442,599,1.0,442.0,1.365344e+05,8.566256e-08,7.0
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.499787,8.236275,3.270351,665,599,4.0,665.0,9.485046e+04,8.566256e-08,5.0
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,10.981421,8.534348,3.270351,58772,599,76.0,88158.0,4.288533e+06,8.566256e-08,220.0
3,Abbey Road,Canning Town,37,599,15428,2228.923167,9.643939,7.709274,3.270351,15428,599,56.0,15428.0,6.130599e+05,8.566256e-08,31.0
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,7.096721,8.807842,3.270351,1208,599,4.0,1208.0,2.337144e+05,8.566256e-08,12.0
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,7.466228,9.048280,3.270351,1748,599,4.0,1748.0,3.723760e+05,8.566256e-08,19.0
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.745236,8.784484,3.270351,850,599,3.0,850.0,1.774013e+05,8.566256e-08,9.0
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.415097,8.283576,3.270351,611,599,3.0,611.0,9.266797e+04,8.566256e-08,5.0
8,Abbey Road,East India,2,599,1522,3384.141666,7.327781,8.126856,3.270351,1522,599,7.0,1522.0,1.588055e+05,8.566256e-08,8.0
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.538140,8.949793,3.270351,691,599,2.0,691.0,1.745056e+05,8.566256e-08,9.0


In [ ]:
def balance_doubly_constrained(pd, orig_field, dest_field, Oi_field, Dj_field, cij_field, beta, 
                               cost_function, Ai_name = "Ai_new", Bj_name = "Bj_new", converge=0.001):
    # Define some variables
    Oi = pd[[orig_field, Oi_field]]
    Dj = pd[[dest_field,Dj_field]]    
    if cost_function.lower() in ['power','pow']:
        beta_cij = np.exp(beta * np.log(pd[cij_field]))
    elif cost_function.lower() in ['exponential','exp']:
        beta_cij = np.exp(beta * pd[cij_field])
    else:
        return "Cost function not specified properly, use 'exp' or 'pow'"
    
    # Create some helper variables
    cnvg = 1
    iteration = 0
    # Now iteratively rebalance the Ai and Bj terms until convergence
    while cnvg > converge:
        if iteration == 0:
            # This first condition sets starting values for Ai and Bj
            # NB sets starting value of Ai assuming Bj is a vector of 1s.
            # We've already established beta_cij with the appropriate cost function, so...
            Oi = Oi.assign(Ai = Dj[Dj_field] * beta_cij)
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Now set up Bjs using starting values of Ai
            Dj = Dj.assign(Bj = Oi['Ai_old'] * Oi[Oi_field] * beta_cij)
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Increment loop
            iteration += 1
        else:
            # This bit is the iterated bit of the loop which refines the values of Ai and Bj
            # First Ai
            Oi['Ai'] = Dj['Bj_old'] * Dj[Dj_field] * beta_cij
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Drop temporary Ai
            Oi.drop('Ai', axis=1, inplace=True)
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True)
            # Calculate the difference between old and new Ais
            Oi['diff'] = np.absolute((Oi['Ai_old'] - Oi['Ai'])/Oi['Ai_old'])
            # Set new Ais to Ai_old
            Oi['Ai_old'] = Oi['Ai']
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Then Bj
            Dj['Bj'] = Oi['Ai_old'] * Oi[Oi_field] * beta_cij
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Drop temporary Bj
            Dj.drop('Bj', axis=1, inplace=True)
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True)
            # Calculate the difference between old and new Bjs
            Dj['diff'] = np.absolute((Dj['Bj_old'] - Dj['Bj'])/Dj['Bj_old'])
            # Set new Bjs to Bj_old
            Dj['Bj_old'] = Dj['Bj']
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Assign higher sum difference from Ai or Bj to cnvg
            cnvg = np.maximum(Oi['diff'].sum(),Dj['diff'].sum())
            
            # Print and increment loop
            print("Iteration:", iteration)
            iteration += 1

    # When the while loop finishes add the computed Ai_old and Bj_old to the dataframe and return
    pd[Ai_name] = Oi['Ai_old']
    pd[Bj_name] = Dj['Bj_old']
    return pd